In [89]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import json

In [104]:
def scroll_method():
    # This method is used to scroll the page
    last_height = 0
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        else:
            last_height = new_height


def detect_type_article(link: str) -> str:
    # This function is used to detect the type of article
    if 'liveblog' in link:
        return 'liveblog'
    elif 'blogs.timesofisrael.com' in link:
        return 'blog'
    else:
        return 'article'

def collect_page_titles() -> list[dict]:
    driver.get('https://www.timesofisrael.com')
    scroll_method()

    result = []
    h = driver.find_elements(By.XPATH, '//div[@class="headline"]/a')
    for link in h:
        result.append({
            'title': link.text, 
            'link': link.get_attribute('href'),
            'type': detect_type_article(link.get_attribute('href')),
        })
    return result

def scrape_liveblog():
    # This function is used to scrape the live blog
    pass

def scrape_article(scrape_object: dict) -> dict:
    driver.get(scrape_object['link'])
    scroll_method()
    try:
        title = driver.find_element(By.XPATH, '//h1[@class="headline"]').text
        content = driver.find_element(By.XPATH, '//div[@class="the-content"]').text
        date = driver.find_element(By.XPATH, '//span[@class="date"]').text
        return {
            'title': title,
            'content': content,
            'date': date,
            'link': scrape_object['link'],
            'type': scrape_object['type'],
        }
    except Exception as e:
        print(f"Error: {e}")
        return None


def scrape_blog():
    pass



driver  = webdriver.Chrome()
titles_to_scrape: list[dict] = collect_page_titles()
with open('result.jsonl', 'w') as f:
    for i in titles_to_scrape:
        f.write(json.dumps(i, ensure_ascii=False) + '\n')
    f.close()
    
driver.quit()

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=136.0.7103.59)
Stacktrace:
#0 0x5633e735778a <unknown>
#1 0x5633e6dfa0a0 <unknown>
#2 0x5633e6de08ae <unknown>
#3 0x5633e6e08be9 <unknown>
#4 0x5633e6e79cef <unknown>
#5 0x5633e6e96c3c <unknown>
#6 0x5633e6e71173 <unknown>
#7 0x5633e6e3dd4b <unknown>
#8 0x5633e6e3e9b1 <unknown>
#9 0x5633e731c93b <unknown>
#10 0x5633e732083a <unknown>
#11 0x5633e7304692 <unknown>
#12 0x5633e73213c4 <unknown>
#13 0x5633e72e94cf <unknown>
#14 0x5633e7345568 <unknown>
#15 0x5633e7345746 <unknown>
#16 0x5633e73565f6 <unknown>
#17 0x7b4aefe9caa4 <unknown>
#18 0x7b4aeff29c3c <unknown>


In [105]:
dicts = []
with open('result.jsonl', 'r') as f:
    for line in f:
        dicts.append(json.loads(line))
    f.close()

driver  = webdriver.Chrome()
for i in dicts:
    if i['type'] == 'article':
        print(i['link'])
        with open('scaped_articles.jsonl', 'a') as f:    
            x = scrape_article(i)
            f.write(json.dumps(x, ensure_ascii=False) + '\n')
            f.close()
driver.quit()


https://www.timesofisrael.com/israel-claims-its-promoting-palestinian-emigration-from-gaza-so-why-are-so-few-leaving/
https://www.timesofisrael.com/two-idf-soldiers-killed-six-wounded-in-southern-gaza-fighting/
https://www.timesofisrael.com/israeli-plan-to-initially-only-feed-60-of-gazans-as-they-endure-extreme-deprivation/
https://www.timesofisrael.com/leading-agency-shuts-its-gaza-soup-kitchens-amid-continued-israeli-aid-ban/
https://www.timesofisrael.com/eu-to-review-trade-ties-with-israel-following-criticism-of-conduct-in-war-in-gaza/
https://www.timesofisrael.com/us-pressuring-humanitarian-groups-to-get-behind-israeli-aid-plan-for-gaza/
https://www.timesofisrael.com/israel-claims-its-promoting-palestinian-emigration-from-gaza-so-why-are-so-few-leaving/
https://www.timesofisrael.com/two-idf-soldiers-killed-six-wounded-in-southern-gaza-fighting/
https://www.timesofisrael.com/israeli-plan-to-initially-only-feed-60-of-gazans-as-they-endure-extreme-deprivation/
https://www.timesofisrae

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=136.0.7103.59)
Stacktrace:
#0 0x5f07c382378a <unknown>
#1 0x5f07c32c60a0 <unknown>
#2 0x5f07c329b310 <unknown>
#3 0x5f07c3345ae8 <unknown>
#4 0x5f07c3362c3c <unknown>
#5 0x5f07c333d173 <unknown>
#6 0x5f07c3309d4b <unknown>
#7 0x5f07c330a9b1 <unknown>
#8 0x5f07c37e893b <unknown>
#9 0x5f07c37ec83a <unknown>
#10 0x5f07c37d0692 <unknown>
#11 0x5f07c37ed3c4 <unknown>
#12 0x5f07c37b54cf <unknown>
#13 0x5f07c3811568 <unknown>
#14 0x5f07c3811746 <unknown>
#15 0x5f07c38225f6 <unknown>
#16 0x705e70c9caa4 <unknown>
#17 0x705e70d29c3c <unknown>


```   author = driver.find_element(By.XPATH, '//span[@class="byline"]/a').text
```
Note to self that author can have several <a> elements if more than one author

In [ ]:
    

driver  = webdriver.Chrome()
scrape_article(dicts[0])

{'title': 'US envoy says Trump-backed Gaza aid mechanism to take effect soon, denies rift with Israel',
 'content': "A US-backed mechanism for distributing aid into Gaza should take effect soon, Washington’s ambassador to Israel said on Friday, ahead of President Donald Trump‘s visit to the Middle East, but he gave few details.\nGaza‘s residents are facing a growing humanitarian crisis with Israel enforcing a months-long blockade on aid supplies to the small Palestinian enclave following the collapse of the most recent ceasefire. Israel has said the Hamas terror group has previously stolen much of the aid entering the territory, using it to maintain control over the population.\nIsrael and the US both indicated in recent days that they were preparing to restore aid through mechanisms that would bypass Hamas.\nAmbassador Mike Huckabee said Friday that several partners had already committed to taking part in the aid arrangement but declined to name them, saying details would be released 